In [2]:
%cd "/content/drive/My Drive/IR/useFiles2"

/content/drive/My Drive/IR/useFiles2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import json
import matplotlib.pyplot as plt
import timeit

In [10]:
def preprocessing():    
    file = open("human_data.txt",'r')
    humandata = file.read()
    humandata = humandata.split()
    count=0
    gene=[]
    classes=[]
    for i in range(0,7):
      classes.append([])
    for lines in humandata[2:]:
      # print(lines)
      if count%2 == 0:
        gene.append(lines)
      else:
        # print(lines)
        y = int(lines)
        classes[y].append(count/2)
      count=count+1
    #print(count/2)
    return gene

In [4]:
def shingling(gene,shingle_index, shingle_size=5):
    #alternative
    #boolean matrix
    #shingle_size=5
    #shingle_index={}
    count=0
    #doc_shingle = np.zeros((1,4380)) 
    doc_shingle= np.array([]).reshape(0,4381)
    #shingle_index is storing the shingle number
    for doc in gene: 
      for i in range(0,int(len(doc)-shingle_size)):
        a = doc[i:i+shingle_size]
        if 'N' in a:
          continue
        if a not in shingle_index:
          shingle_index[a] = len(shingle_index)
          doc_shingle = np.vstack ((doc_shingle,np.zeros((1,4381)))) # corresponding shingle 4380 row
          doc_shingle[shingle_index[a]][count] = 1
        else:
          doc_shingle[shingle_index[a]][count] = 1
      count+=1
    np.savetxt("doc_shingle.txt",doc_shingle)
    # with open("shingles.txt","w") as file:
    #   json.dump(shingle_index,file)

In [6]:
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

In [5]:
def minHash(doc_shingle):
    start = timeit.default_timer()
    min_hash = np.zeros((1,doc_shingle.shape[1]))
    for i in range(0,100):
      np.random.shuffle(doc_shingle)
      # list.append(doc_shingle)
      min_hash = np.vstack ((min_hash, first_nonzero(doc_shingle,0)))
    min_hash = np.delete(min_hash,0,axis=0)
    end = timeit.default_timer()
    print("Time taken to create Min Hash : ", end-start)
    return min_hash
    # print(min_hash)

In [7]:
def query_hashing(query,shingle_index,shingle_size):
    query_shingles=np.zeros((len(shingle_index),1))
    for i in range(0,len(query)-shingle_size):
      a = query[i:i+shingle_size]
      if a in shingle_index:
        query_shingles[int(shingle_index[a])][0]=1
    return query_shingles

In [ ]:
def Bucketing(a,b,min_hash,r):
    buckets={}
    doc_size=min_hash.shape[1]-1
    for docs in range(0,doc_size):
      temp_sum=0
      for rows in range(0,r):
        temp_sum+=min_hash[docs][rows]
      temp_sum=(a*temp_sum+b)%doc_size
      if temp_sum in buckets:
        buckets[temp_sum].append(docs)
      else:
        buckets[temp_sum]=set()
        buckets[temp_sum].append(docs)
    return buckets

In [ ]:
def Similarity(min_hash,r,b):
  query=min_hash[:,min_hash.shape[1]-1]
  hash_random=np.random.randint(100,size=(2,b))
  for bands in range(0,b):
    buckets=Bucketing(hash_random[0][bands],hash_random[1][bands],min_hash,r)
    

In [ ]:
def JaccardIndex(min_hash):
  r,c = np.shape(min_hash)
  row=10
  band=10
  #print(type(band))
  freq=np.zeros(c-1)
  query_col=min_hash[:,c-1]
  for bands in range(0,band):
    query=min_hash[bands:bands+row,c-1]
    for cols in range(0,c-1):
      if np.any(min_hash[bands:bands+row,cols]==query):
        freq[cols]+=1
        #print(cols)
  freq=(freq/band)
  #final = np.where(freq>0.95)
  return freq

In [15]:
def query_processing(query,shingle_index,shingle_size):
  doc_shingle=np.loadtxt("doc_shingle.txt")
  query_shingle=query_hashing(query,shingle_index,shingle_size)
  # print(query_shingle.shape[0],query_shingle.shape[1],doc_shingle.shape[0],doc_shingle.shape[1])
  doc_shingle=np.concatenate((doc_shingle,query_shingle),axis=1)
  min_hash=minHash(doc_shingle)
  print(min_hash)
  print(np.argmax(JaccardIndex(min_hash)))

In [16]:
# gene=preprocessing()
# shingle_size=int(input("Enter shingle size: "))
# shingle_index={}
# shingling(gene,shingle_index,shingle_size)
while True:
  query = input("Press Quit to exit. Enter Query : ")
  if query == "Quit":
    break
  else:
    query_processing(query,shingle_index,shingle_size)

Press Quit to exit. Enter Query : ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACCACCTACCTCCCTCACCAAAGCCCATAAAAATAAAAAATTATAACAAACCCTGAGAACCAAAATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG
Time taken to create Min Hash :  1.815297778999991
[[ 4.  2.  1. ...  6. -1.  4.]
 [ 4.  4.  0. ...  1. -1.  4.]
 [11.  2.  2. ...  1. -1. 11.]
 ...
 [ 4.  1.  0. ...  2. -1.  4.]
 [19.  3.  0. ...  1. -1. 19.]
 [ 2.  0.  0. ...  3. -1.  2.]]


NameError: ignored

In [13]:
print(shingle_index)

{'ATGCC': 0, 'TGCCC': 1, 'GCCCC': 2, 'CCCCA': 3, 'CCCAA': 4, 'CCAAC': 5, 'CAACT': 6, 'AACTA': 7, 'ACTAA': 8, 'CTAAA': 9, 'TAAAT': 10, 'AAATA': 11, 'AATAC': 12, 'ATACT': 13, 'TACTA': 14, 'ACTAC': 15, 'CTACC': 16, 'TACCG': 17, 'ACCGT': 18, 'CCGTA': 19, 'CGTAT': 20, 'GTATG': 21, 'TATGG': 22, 'ATGGC': 23, 'TGGCC': 24, 'GGCCC': 25, 'GCCCA': 26, 'CCCAC': 27, 'CCACC': 28, 'CACCA': 29, 'ACCAT': 30, 'CCATA': 31, 'CATAA': 32, 'ATAAT': 33, 'TAATT': 34, 'AATTA': 35, 'ATTAC': 36, 'TTACC': 37, 'TACCC': 38, 'ACCCC': 39, 'CCCCC': 40, 'CCCAT': 41, 'CATAC': 42, 'TACTC': 43, 'ACTCC': 44, 'CTCCT': 45, 'TCCTT': 46, 'CCTTA': 47, 'CTTAC': 48, 'TTACA': 49, 'TACAC': 50, 'ACACT': 51, 'CACTA': 52, 'ACTAT': 53, 'CTATT': 54, 'TATTC': 55, 'ATTCC': 56, 'TTCCT': 57, 'TCCTC': 58, 'CCTCA': 59, 'CTCAT': 60, 'TCATC': 61, 'CATCA': 62, 'ATCAC': 63, 'TCACC': 64, 'CACCC': 65, 'ACCCA': 66, 'TAAAA': 67, 'AAAAA': 68, 'AAAAT': 69, 'AATAT': 70, 'ATATT': 71, 'TATTA': 72, 'ATTAA': 73, 'TTAAA': 74, 'TAAAC': 75, 'AAACA': 76, 'AACAC':

In [ ]:
query = input("Enter the query:")
gene.append(query)

Enter the query:ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACCACCTACCTCCCTCACCAAAGCCCATAAAAATAAAAAATTATAACAAACCCTGAGAACCAAAATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG    


In [ ]:
r,c = np.shape(min_hash)
for i in range(0,r):
  str=""
  opt =["%0.0f" % i for i in min_hash[i]]
  for j in range(0,c):
    str = str + opt[j]+" "
  print(str)

1 1 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 13 1 0 2 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 1 6 1 104 0 0 18 4 4 2 2 4 4 4 4 2 2 18 18 6 2 4 2 2 3 3 3 2 2 2 2 2 3 4 0 0 2 0 0 0 2 13 0 0 0 0 0 0 0 2 4 0 0 0 0 0 9 9 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 9 3 0 0 0 0 14 17 17 9 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 20 20 1 20 20 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 18 4 0 1 0 2 0 0 2 2 0 0 0 0 0 0 0 0 1 0 0 0 0 3 3 0 0 0 1 0 0 9 15 15 6 15 15 15 0 0 0 0 0 0 0 0 2 3 3 3 0 0 0 0 0 0 18 0 5 5 5 1 0 0 0 0 6 0 6 0 0 3 1 6 1 1 6 2 3 3 3 3 0 0 0 0 0 0 0 3 0 0 0 2 3 2 2 1 3 2 1 2 13 2 3 3 2 13 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 1 3 1 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 13 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 20 0 0 0 0 0 0 0 0 3 0 0 4 1 1 0 3 4 2 0 0 1 0 0 3 0 2 2 3 2 3 2 0 0 0 0 0 0 0 0 0 4 1 1 3 3 3 3 0 3 3 3 1 1 0 0 0 0 0 3 3 0 9 0 0 0 3 4 17 4 4 1 0 0 0 0 0 0 2 2 3 17 17 15 17 15 15 15 15 2 0 0 0 4 7 0 0 0 0 0 0 2 0 0 2 2 0 1 1 1 3 63 2 2 2 2 7 6 0 2 0 0 0 0 4 4 0 0 0 0 0 0 2 0 0 1 0

In [ ]:
print(r," ",c)

100   4381


In [ ]:
final=JaccardIndex(min_hash)
x=np.shape(final)[0]
for i in range(0,x):
  print(final[i])

<class 'int'>
1.0
1.0
0.7
0.6
0.7
0.7
0.7
0.7
0.7
1.0
1.0
1.0
0.9
0.0
0.0
0.7
0.6
0.6
0.6
0.6
0.6
0.6
0.6
1.0
0.6
0.7
0.7
1.0
0.9
0.7
0.7
0.7
0.7
0.7
0.0
0.0
0.7
1.0
1.0
1.0
0.6
0.6
1.0
0.6
0.6
1.0
0.8
0.6
0.7
0.0
0.7
0.7
0.6
0.8
0.8
1.0
1.0
0.8
1.0
0.7
0.7
1.0
1.0
0.8
0.6
1.0
1.0
1.0
1.0
1.0
0.5
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.4
0.6
1.0
1.0
0.8
1.0
1.0
1.0
0.6
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.8
1.0
1.0
1.0
0.8
0.8
1.0
1.0
0.6
0.6
0.6
0.6
0.7
0.7
0.7
0.6
1.0
0.6
1.0
0.6
0.5
0.5
0.7
0.7
0.7
0.7
0.7
0.7
0.0
0.0
0.7
0.7
0.7
0.0
0.6
0.0
0.9
0.0
0.7
1.0
0.6
0.6
1.0
0.7
1.0
1.0
1.0
1.0
1.0
1.0
0.6
0.6
1.0
1.0
0.6
1.0
1.0
0.6
1.0
1.0
0.6
0.6
1.0
0.6
1.0
1.0
0.6
0.6
0.8
0.4
0.7
0.7
1.0
1.0
0.6
1.0
1.0
1.0
1.0
0.6
1.0
0.7
1.0
1.0
0.5
1.0
1.0
1.0
0.8
0.9
1.0
0.8
1.0
1.0
1.0
0.8
0.0
0.0
0.9
0.8
0.9
0.8
0.8
0.7
0.7
0.7
0.6
0.7
1.0
1.0
0.7
0.4
0.6
1.0
0.7
1.0
1.0
0.7
0.6
0.5
0.8
0.8
0.4
0.0
0.8
0.8
0.6
0.6
0.6
0.6
1.0
1.0
0.7
0.7
0.7
0.8
0.7
0.7
0.5
0.5
0.4
0.4
0.4
0.4
0.0
0.4
1.0
1.0
1.0
1.0
1.

In [ ]:
import numpy as np
np.array([]).reshape(0,2)

    

array([], shape=(0, 2), dtype=float64)

In [ ]:
#1 — (1 — J^n)^b
print((1-(0.95)**5)**20)

1.2319241384161333e-13


In [ ]:
import matplotlib.pyplot as plt
def create_plot():
  x=np.arange(0,1,0.001)
  y=(1-(1-(x)**20)**5)
  return (x,y)
def plot_points():
  
fig = plt.figure()
x,y=create_plot()
plt1 = fig.add_subplot(221)
plt1.plot(x,y,color ='g')
plt.show()

IndentationError: ignored

In [ ]:
import random
for i in range(1):
        dnaSequence = ""
        for _ in range(random.randint(0, 49)):
            dnaSequence += genDNA(random.randint(0,3))